In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

# 基础神经网络

本章将开启一段深入探究之旅，我们将深入研究前几章中使用的模型的内部结构。我们会涉及许多之前已经见过的内容，但这一次我们将更加细致地观察实现细节，而对实际问题——如何以及为什么事物是现在这个样子——的关注则会相对较少。

我们将从零开始构建一切，仅使用张量的基本索引。我们将从头开始编写一个神经网络，然后手动实现反向传播，以便我们清楚地了解在调用`loss.backward`时PyTorch中究竟发生了什么。我们还将看到如何通过自定义的*autograd*函数扩展PyTorch，这允许我们指定自己的前向和后向计算。

## 从头开始构建神经网络层

让我们首先回顾一下矩阵乘法在基本神经网络中的使用。由于我们将从零开始构建一切，最初我们将只使用纯Python（除了对PyTorch张量的索引），一旦我们了解了如何创建它，我们就会用PyTorch的功能来替换纯Python。

### 建模神经元

一个神经元接收一定数量的输入，并为每个输入设定一个内部权重。它将这些加权输入求和以产生输出，并加上一个内部偏置。在数学上，这可以表示为：

$$ out = \sum_{i=1}^{n} x_{i} w_{i} + b $$

如果我们将输入命名为 $(x_{1},\dots,x_{n})$，权重为 $(w_{1},\dots,w_{n})$，偏置为 $b$。在代码中，这可以转化为：

```python
output = sum([x*w for x,w in zip(inputs,weights)]) + bias
```

然后，这个输出在发送到另一个神经元之前，会通过一个非线性函数，称为*激活函数*。在深度学习中，最常见的激活函数是*修正线性单元*（ReLU），正如我们所看到的，这是一种花哨的说法：

```python
def relu(x): return x if x >= 0 else 0
```

深度学习模型是通过堆叠许多这样的神经元在连续的层中构建的。我们创建一个具有一定数量神经元的第一层（称为*隐藏层大小*），并将所有输入连接到这些神经元上。这样的层通常被称为*全连接层*或*密集层*（因为密集连接），或者*线性层*。

它需要计算，对于我们批次中的每个`input`和每个具有给定`weight`的神经元，它们的点积：

```python
sum([x*w for x,w in zip(input,weight)])
```

如果你有一点线性代数的知识，你可能记得当你进行大量的这种点积时，就是在进行*矩阵乘法*。更准确地说，如果我们的输入在一个矩阵`x`中，大小为`batch_size`乘以`n_inputs`，并且如果我们将神经元的权重分组在一个大小为`n_neurons`乘以`n_inputs`的矩阵`w`中（每个神经元必须有与其输入数量相同的权重），以及所有偏置在一个大小为`n_neurons`的向量`b`中，那么这个全连接层的输出就是：

```python
y = x @ w.t() + b
```

这里的`@`代表矩阵乘法，`w.t()`是`w`的转置矩阵。输出`y`的大小为`batch_size`乘以`n_neurons`，在位置`(i,j)`处我们有（对于喜欢数学的人来说）：

$$y_{i,j} = \sum_{k=1}^{n} x_{i,k} w_{k,j} + b_{j}$$

或者在代码中：

```python
y[i,j] = sum([a * b for a,b in zip(x[i,:],w[j,:])]) + b[j]
```

转置是必要的，因为在矩阵乘法`m @ n`的数学定义中，系数`(i,j)`是：

```python
sum([a * b for a,b in zip(m[i,:],n[:,j])])
```

所以我们需要的基本操作是矩阵乘法，因为它隐藏在神经网络的核心。

### 从头开始矩阵乘法

让我们编写一个函数来计算两个张量的矩阵乘积，在允许自己使用PyTorch版本的它之前。我们将只使用PyTorch张量的索引功能：

In [ ]:
import torch
from torch import tensor

我们需要三个嵌套的`for`循环：一个用于行索引，一个用于列索引，还有一个用于内部求和。`ac`和`ar`分别代表`a`的列数和`a`的行数（同样的约定也用于`b`），我们通过检查`a`的列数是否与`b`的行数相等，来确保计算矩阵乘积是可能的：

In [ ]:
def matmul(a,b):
    ar,ac = a.shape # n_rows * n_cols
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac): c[i,j] += a[i,k] * b[k,j]
    return c

为了测试这个功能，我们将假装（使用随机矩阵）我们正在处理一小批5张MNIST图像，将它们展平成28×28的向量，并通过线性模型将它们转换成10个激活值：

In [ ]:
m1 = torch.randn(5,28*28)
m2 = torch.randn(784,10)

让我们使用Jupyter的“魔法”命令`%time`来计时我们的函数：

In [ ]:
%time t1=matmul(m1, m2)

CPU times: user 1.15 s, sys: 4.09 ms, total: 1.15 s
Wall time: 1.15 s


然后看看这与PyTorch内置的`@`操作符相比如何：

In [ ]:
%timeit -n 20 t2=m1@m2

14 µs ± 8.95 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)


正如我们所见，在Python中使用三个嵌套循环是一个非常糟糕的想法！Python是一种运行速度较慢的语言，这样做效率不会很高。我们在这里看到，PyTorch的速度大约比Python快100,000倍——而这还是在我们甚至还没有开始使用GPU的情况下！

这种差异来自哪里？PyTorch并没有用Python编写其矩阵乘法，而是用C++编写以提高速度。一般来说，每当我们在张量上进行计算时，我们需要将它们*向量化*，以便我们可以利用PyTorch的速度，通常是通过使用两种技术：逐元素算术和广播。

### 元素运算

所有的基本运算符（`+`、`-`、`*`、`/`、`>`、`<`、`==`）都可以逐元素应用。这意味着，如果我们对形状相同的两个张量`a`和`b`执行`a+b`，我们将得到一个张量，它由`a`和`b`的元素相加的结果组成：

In [ ]:
a = tensor([10., 6, -4])
b = tensor([2., 8, 7])
a + b

tensor([12., 14.,  3.])

布尔运算符将返回一个布尔值数组：

In [ ]:
a < b

tensor([False,  True,  True])

如果我们想知道`a`的每个元素是否都小于`b`中对应的元素，或者两个张量是否相等，我们需要将这些逐元素操作与`torch.all`结合起来：

In [ ]:
(a < b).all(), (a==b).all()

(tensor(False), tensor(False))

像`all()`、`sum()`和`mean()`这样的缩减操作返回的张量只有一个元素，称为秩为0的张量。如果你想将其转换为普通的Python布尔值或数字，你需要调用`.item()`：

In [ ]:
(a + b).mean().item()

9.666666984558105

逐元素操作适用于任何秩的张量，只要它们的形状相同：

In [ ]:
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])
m*m

tensor([[ 1.,  4.,  9.],
        [16., 25., 36.],
        [49., 64., 81.]])

然而，你不能在形状不同的张量上执行逐元素操作（除非它们是可广播的，这将在下一节中讨论）：

In [ ]:
n = tensor([[1., 2, 3], [4,5,6]])
m*n

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 0

通过逐元素算术，我们可以移除我们的三个嵌套循环中的一个：我们可以在求和所有元素之前，先乘以对应于`a`的第`i`行和`b`的第`j`列的张量，这将加速计算，因为内部循环现在将以C语言的速度由PyTorch执行。

要访问一列或一行，我们可以简单地写成`a[i,:]`或`b[:,j]`。`:`表示在该维度上取所有元素。我们可以通过传递一个范围，如`1:5`，而不是仅仅使用`:`，来限制这一点，并只取该特定维度的一个切片。在这种情况下，我们将取第1到第4列（第二个数字是不包含的）的元素。

一个简化是，我们总是可以省略末尾的冒号，所以`a[i,:]`可以缩写为`a[i]`。考虑到所有这些，我们可以编写我们矩阵乘法的新版本：

In [ ]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc): c[i,j] = (a[i] * b[:,j]).sum()
    return c

In [ ]:
%timeit -n 20 t3 = matmul(m1,m2)

1.7 ms ± 88.1 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)


仅仅通过移除那个内部的`for`循环，我们的速度已经提高了大约700倍！而这仅仅是开始——通过广播，我们可以移除另一个循环，获得更加重要的速度提升。

### 广播

正如我们在<<chapter_mnist_basics>>中讨论的，广播是由[NumPy库](https://docs.scipy.org/doc/)引入的一个术语，它描述了在算术运算中如何处理不同秩的张量。例如，显然没有办法将一个3×3矩阵与一个4×5矩阵相加，但是如果我们想要将一个标量（可以表示为1×1张量）与一个矩阵相加呢？或者将一个大小为3的向量与一个3×4矩阵相加？在这两种情况下，我们都能找到一种方法来理解这个操作。

广播提供了特定的规则来编码在尝试进行逐元素操作时形状何时兼容，以及如何扩展较小形状的张量以匹配较大形状的张量。如果你想编写能够快速执行的代码，掌握这些规则是至关重要的。在本节中，我们将扩展我们之前对广播的处理，以理解这些规则。

#### 用标量进行广播

使用标量的广播是最简单的广播类型。当我们有一个张量`a`和一个标量时，我们只需想象一个与`a`形状相同的张量，用那个标量填充，然后执行操作：

In [ ]:
a = tensor([10., 6, -4])
a > 0

tensor([ True,  True, False])

我们如何能够进行这种比较？`0`被*广播*以具有与`a`相同的维度。请注意，这是在不创建一个充满零的张量存储在内存中的情况下完成的（那将非常低效）。

如果你想要通过从整个数据集（一个矩阵）中减去均值（一个标量）并除以标准差（另一个标量）来标准化你的数据集，这非常有用：

In [ ]:
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])
(m - 5) / 2.73

tensor([[-1.4652, -1.0989, -0.7326],
        [-0.3663,  0.0000,  0.3663],
        [ 0.7326,  1.0989,  1.4652]])

如果矩阵的每一行有不同的均值怎么办？在这种情况下，你需要将一个向量广播到一个矩阵。

#### 将向量广播到矩阵

我们可以按照以下方式将一个向量广播到一个矩阵：

In [ ]:
c = tensor([10.,20,30])
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])
m.shape,c.shape

(torch.Size([3, 3]), torch.Size([3]))

In [ ]:
m + c

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

在这里，`c`的元素被扩展以形成三行以匹配，使得操作成为可能。同样，PyTorch实际上并不会在内存中创建`c`的三个副本。这是通过`expand_as`方法在幕后完成的：

In [ ]:
c.expand_as(m)

tensor([[10., 20., 30.],
        [10., 20., 30.],
        [10., 20., 30.]])

如果我们查看相应的张量，我们可以请求它的`storage`属性（它显示用于张量的实际内存内容），以检查没有存储无用的数据：

In [ ]:
t = c.expand_as(m)
t.storage()

 10.0
 20.0
 30.0
[torch.FloatStorage of size 3]

尽管张量官方上有九个元素，但实际上只有三个标量存储在内存中。这得益于给该维度一个*步长*为0的巧妙技巧（这意味着当PyTorch通过添加步长寻找下一行时，它不会移动）：

In [ ]:
t.stride(), t.shape

((0, 1), torch.Size([3, 3]))

由于`m`的大小为3×3，广播有两种方式。在最后一个维度上进行广播是一个来自广播规则的惯例，这与我们排列张量的方式无关。如果我们这样做，我们会得到相同的结果：

In [ ]:
c + m

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

实际上，只有当向量的大小为`n`且矩阵的大小为`m`×`n`时，才能将向量广播到矩阵。

In [ ]:
c = tensor([10.,20,30])
m = tensor([[1., 2, 3], [4,5,6]])
c+m

tensor([[11., 22., 33.],
        [14., 25., 36.]])

这将无法正常运行：

In [ ]:
c = tensor([10.,20])
m = tensor([[1., 2, 3], [4,5,6]])
c+m

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

如果我们想要在另一个维度上进行广播，我们必须改变我们的向量的形状，使其成为一个3×1的矩阵。这可以通过PyTorch中的`unsqueeze`方法来实现：

In [ ]:
c = tensor([10.,20,30])
m = tensor([[1., 2, 3], [4,5,6], [7,8,9]])
c = c.unsqueeze(1)
m.shape,c.shape

(torch.Size([3, 3]), torch.Size([3, 1]))

这一次，`c`在列方向上被扩展：

In [ ]:
c+m

tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 38., 39.]])

和之前一样，内存中只存储了三个标量：

In [ ]:
t = c.expand_as(m)
t.storage()

 10.0
 20.0
 30.0
[torch.FloatStorage of size 3]

并且扩展后的张量具有正确的形状，因为列维度的步长为0：

In [ ]:
t.stride(), t.shape

((1, 0), torch.Size([3, 3]))

使用广播，默认情况下，如果我们需要添加维度，它们会被添加在最前面。当我们之前进行广播时，PyTorch在幕后执行了`c.unsqueeze(0)`：

In [ ]:
c = tensor([10.,20,30])
c.shape, c.unsqueeze(0).shape,c.unsqueeze(1).shape

(torch.Size([3]), torch.Size([1, 3]), torch.Size([3, 1]))

`unsqueeze`命令可以用`None`索引来替代：

In [ ]:
c.shape, c[None,:].shape,c[:,None].shape

(torch.Size([3]), torch.Size([1, 3]), torch.Size([3, 1]))

你总是可以省略末尾的冒号，而`...`表示所有前面的维度：

In [ ]:
c[None].shape,c[...,None].shape

(torch.Size([1, 3]), torch.Size([3, 1]))

有了这个，我们可以在我们的矩阵乘法函数中移除另一个`for`循环。现在，我们不是将`a[i]`与`b[:,j]`相乘，而是可以使用广播将`a[i]`与整个矩阵`b`相乘，然后对结果求和：

In [ ]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
#       c[i,j] = (a[i,:]          * b[:,j]).sum() # previous
        c[i]   = (a[i  ].unsqueeze(-1) * b).sum(dim=0)
    return c

In [ ]:
%timeit -n 20 t4 = matmul(m1,m2)

357 µs ± 7.2 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)


我们现在的速度比我们最初的实现快了3,700倍！在我们继续之前，让我们稍微详细地讨论一下广播的规则。

#### 广播规则

当对两个张量进行操作时，PyTorch会逐元素比较它们的形状。它从*尾部维度*开始，向后工作，遇到空维度时添加1。两个维度在以下情况下是*兼容*的：

- 它们相等。
- 其中一个是1，在这种情况下，该维度被广播以使其与其他维度相同。

数组不需要具有相同数量的维度。例如，如果你有一个256×256×3的RGB值数组，并且你想通过不同的值缩放图像中的每种颜色，你可以将图像与一个有三个值的一维数组相乘。根据广播规则对齐这些数组的尾部轴的大小，可以看出它们是兼容的：

```
图像  (3维张量): 256 x 256 x 3
缩放  (1维张量): (1)   (1)  3
结果 (3维张量): 256 x 256 x 3
```

然而，一个大小为256×256的2D张量与我们的图像不兼容：

```
图像  (3维张量): 256 x 256 x   3
缩放  (2维张量):  (1)  256 x 256
错误
```

在我们之前的例子中，我们有一个3×3的矩阵和一个大小为3的向量，广播是在行上进行的：

```
矩阵 (2维张量):   3 x 3
向量 (1维张量): (1)   3
结果 (2维张量):   3 x 3
```

作为一个练习，尝试确定当你需要对大小为`64 x 3 x 256 x 256`的图像批次进行标准化时，需要添加哪些维度（以及在哪里），使用三个元素的向量（一个用于均值，一个用于标准差）。

另一种简化张量操作的有用方法是使用爱因斯坦求和约定。

### 爱因斯坦求和

在使用PyTorch操作`@`或`torch.matmul`之前，我们还有一种实现矩阵乘法的方法：爱因斯坦求和（`einsum`）。这是一种紧凑的表示方法，用于以一般方式结合乘积和求和。我们这样写方程：

```
ik,kj -> ij
```

左侧代表操作数的维度，用逗号分隔。这里我们有两个张量，每个张量都有两个维度（`i,k`和`k,j`）。右侧代表结果的维度，所以这里我们有一个两个维度`i,j`的张量。

爱因斯坦求和记号的规则如下：

1. 如果左侧的重复索引不在右侧，则隐含地对它们进行求和。
2. 每个索引最多在左侧出现两次。
3. 左侧未重复的索引必须出现在右侧。

所以在我们的示例中，由于`k`是重复的，我们对那个索引进行求和。最终，这个公式代表了当我们在`(i,j)`处放入第一个张量中`(i,k)`系数与第二个张量中`(k,j)`系数乘积的所有系数之和时得到的矩阵...这正是矩阵乘积！下面是我们如何在PyTorch中编码这个的示例：

In [ ]:
def matmul(a,b): return torch.einsum('ik,kj->ij', a, b)

爱因斯坦求和是一种非常实用的方法，用于表达涉及索引和乘积求和的操作。请注意，你可以在左侧只有一个成员。例如，这个：

```python
torch.einsum('ij->ji', a)
```

将返回矩阵`a`的转置。你也可以有三个或更多的成员。这个：

```python
torch.einsum('bi,ij,bj->b', a, b, c)
```

将返回一个大小为`b`的向量，其中第`k`个坐标是`a[k,i] b[i,j] c[k,j]`的和。当你有更多维度，特别是因为批次而有更多维度时，这种表示法特别方便。例如，如果你有两个批次的矩阵，并且想要计算每个批次的矩阵乘积，你可以这样做：

```python
torch.einsum('bik,bkj->bij', a, b)
```

让我们回到使用`einsum`的新`matmul`实现，并看看它的执行速度：

In [ ]:
%timeit -n 20 t5 = matmul(m1,m2)

68.7 µs ± 4.06 µs per loop (mean ± std. dev. of 7 runs, 20 loops each)


正如你所见，它不仅实用，而且非常快。`einsum`通常是在PyTorch中执行自定义操作的最快方式，无需深入C++和CUDA。（但它通常不如精心优化的CUDA代码快，正如你在“从头开始的矩阵乘法”的结果中看到的那样。）

现在我们知道了如何从头实现矩阵乘法，我们准备好使用仅矩阵乘法来构建我们的神经网络——特别是它的前向和后向传播。

## 前向和后向传播

正如我们在<<chapter_mnist_basics>>中看到的，为了训练一个模型，我们需要计算给定损失相对于其参数的所有梯度，这被称为*后向传播*。*前向传播*是我们根据矩阵乘积计算模型在给定输入上的输出。当我们定义我们的第一个神经网络时，我们还将深入探讨正确初始化权重的问题，这对于使训练正确开始至关重要。

### 定义和初始化一个层

我们将首先以一个两层神经网络为例。正如我们所看到的，一个层可以表示为`y = x @ w + b`，其中`x`是我们的输入，`y`是我们的输出，`w`是层的权重（如果我们不像之前那样转置，它的大小是输入数量乘以神经元数量），而`b`是偏置向量：

In [ ]:
def lin(x, w, b): return x @ w + b

我们可以将第二层叠加在第一层之上，但由于数学上两个线性操作的组合是另一个线性操作，这只有在我们在中间放入某种非线性的东西才有意义，这被称为激活函数。正如本章开头提到的，在深度学习应用中，最常用的激活函数是ReLU，它返回`x`和`0`中的最大值。

我们实际上不会在本章训练我们的模型，所以我们将使用随机张量作为我们的输入和目标。假设我们的输入是200个大小为100的向量，我们将它们组合成一个批次，我们的目标是200个随机浮点数：

In [ ]:
x = torch.randn(200, 100)
y = torch.randn(200)

对于我们的两层模型，我们需要两个权重矩阵和两个偏置向量。假设我们有一个隐藏层大小为50，输出大小为1（对于我们的一个输入，在本玩具示例中，相应的输出是一个浮点数）。我们随机初始化权重，并将偏置设置为零：

In [ ]:
w1 = torch.randn(100,50)
b1 = torch.zeros(50)
w2 = torch.randn(50,1)
b2 = torch.zeros(1)

那么我们第一层的结果就是：

In [ ]:
l1 = lin(x, w1, b1)
l1.shape

torch.Size([200, 50])

请注意，这个公式适用于我们的输入批次，并返回一个隐藏状态的批次：`l1`是一个大小为200（我们的批次大小）乘以50（我们的隐藏大小）的矩阵。

然而，我们的模型初始化方式存在问题。为了理解这个问题，我们需要看看`l1`的均值和标准差（std）：

In [ ]:
l1.mean(), l1.std()

(tensor(0.0019), tensor(10.1058))

均值接近零，这是可以理解的，因为我们的输入和权重矩阵的均值都接近零。但是，标准差代表了我们的激活值离均值有多远，它从1变到了10。这是一个非常大的问题，因为这仅仅是在一层的情况下。现代神经网络可能有数百层，所以如果每一层都将我们的激活值的规模乘以10，那么在最后一层结束时，我们将不会有计算机能够表示的数字。

实际上，如果我们在`x`和大小为100×100的随机矩阵之间进行50次乘法，我们将得到：

In [ ]:
x = torch.randn(200, 100)
for i in range(50): x = x @ torch.randn(100,100)
x[0:5,0:5]

tensor([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]])

结果是到处都是`nan`。所以也许我们矩阵的规模太大了，我们需要有更小的权重？但是，如果我们使用太小的权重，我们将会遇到相反的问题——我们的激活值的规模将从1变为0.1，经过50层之后，我们将到处都是零：

In [ ]:
x = torch.randn(200, 100)
for i in range(50): x = x @ (torch.randn(100,100) * 0.01)
x[0:5,0:5]

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

因此，我们必须精确地调整我们的权重矩阵的规模，以便我们的激活值的标准差保持在1。我们可以数学上计算出确切的值，正如Xavier Glorot和Yoshua Bengio在["Understanding the Difficulty of Training Deep Feedforward Neural Networks"](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)中所说明的。给定层的正确规模是$1/\sqrt{n_{in}}$，其中$n_{in}$代表输入的数量。

在我们的情况下，如果我们有100个输入，我们应该将我们的权重矩阵按0.1的比例缩放：

In [ ]:
x = torch.randn(200, 100)
for i in range(50): x = x @ (torch.randn(100,100) * 0.1)
x[0:5,0:5]

tensor([[ 0.7554,  0.6167, -0.1757, -1.5662,  0.5644],
        [-0.1987,  0.6292,  0.3283, -1.1538,  0.5416],
        [ 0.6106,  0.2556, -0.0618, -0.9463,  0.4445],
        [ 0.4484,  0.7144,  0.1164, -0.8626,  0.4413],
        [ 0.3463,  0.5930,  0.3375, -0.9486,  0.5643]])

最终得到的数字既不是零也不是`nan`！请注意，即使经过这50个假层之后，我们的激活值的规模是多么稳定：

In [ ]:
x.std()

tensor(0.7042)

如果你稍微调整一下缩放值，你会注意到即使是从0.1的微小变化也会导致你得到非常小或非常大的数字，所以正确初始化权重是极其重要的。

让我们回到我们的神经网络。由于我们稍微调整了我们的输入，我们需要重新定义它们：

In [ ]:
x = torch.randn(200, 100)
y = torch.randn(200)

对于我们的权重，我们将使用正确的缩放值，这被称为*Xavier初始化*（或*Glorot初始化*）：

In [ ]:
from math import sqrt
w1 = torch.randn(100,50) / sqrt(100)
b1 = torch.zeros(50)
w2 = torch.randn(50,1) / sqrt(50)
b2 = torch.zeros(1)

现在，如果我们计算第一层的结果，我们可以检查均值和标准差都在控制之下：

In [ ]:
l1 = lin(x, w1, b1)
l1.mean(),l1.std()

(tensor(-0.0050), tensor(1.0000))

非常好。现在我们需要通过一个ReLU激活函数，所以让我们定义一个。ReLU移除负值并将它们替换为零，这也可以看作是将我们的张量限制在零：

In [ ]:
def relu(x): return x.clamp_min(0.)

我们将我们的激活值通过这个函数：

In [ ]:
l2 = relu(l1)
l2.mean(),l2.std()

(tensor(0.3961), tensor(0.5783))

我们回到了起点：我们的激活值的均值变为了0.4（这是可以理解的，因为我们移除了负值），标准差下降到了0.58。所以就像之前一样，经过几层之后，我们可能会最终得到零：

In [ ]:
x = torch.randn(200, 100)
for i in range(50): x = relu(x @ (torch.randn(100,100) * 0.1))
x[0:5,0:5]

tensor([[0.0000e+00, 1.9689e-08, 4.2820e-08, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.6701e-08, 4.3501e-08, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.0976e-08, 3.0411e-08, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.8457e-08, 4.9469e-08, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.9949e-08, 4.1643e-08, 0.0000e+00, 0.0000e+00]])

这意味着我们的初始化方法不正确。为什么？在Glorot和Bengio撰写他们的文章时，神经网络中流行的激活函数是双曲正切（tanh，也就是他们使用的激活函数），而那种初始化方法并没有考虑到我们的ReLU。幸运的是，已经有人为我们做了数学计算，并为我们计算出了正确的缩放比例。在["Delving Deep into Rectifiers: Surpassing Human-Level Performance"](https://arxiv.org/abs/1502.01852)（我们已经看过这篇文章——它是引入ResNet的文章）中，Kaiming He等人展示了我们应该使用以下缩放比例：$\sqrt{2 / n_{in}}$，其中$n_{in}$是我们模型的输入数量。让我们看看这给我们带来了什么结果：

In [ ]:
x = torch.randn(200, 100)
for i in range(50): x = relu(x @ (torch.randn(100,100) * sqrt(2/100)))
x[0:5,0:5]

tensor([[0.2871, 0.0000, 0.0000, 0.0000, 0.0026],
        [0.4546, 0.0000, 0.0000, 0.0000, 0.0015],
        [0.6178, 0.0000, 0.0000, 0.0180, 0.0079],
        [0.3333, 0.0000, 0.0000, 0.0545, 0.0000],
        [0.1940, 0.0000, 0.0000, 0.0000, 0.0096]])

这样好多了：这次我们的数字没有全部变成零。所以让我们回到我们神经网络的定义，并使用这种初始化方法（它被称为*Kaiming初始化*或*He初始化*）：

In [ ]:
x = torch.randn(200, 100)
y = torch.randn(200)

In [ ]:
w1 = torch.randn(100,50) * sqrt(2 / 100)
b1 = torch.zeros(50)
w2 = torch.randn(50,1) * sqrt(2 / 50)
b2 = torch.zeros(1)

让我们看看在经过第一层线性层和ReLU之后，我们的激活值的规模：

In [ ]:
l1 = lin(x, w1, b1)
l2 = relu(l1)
l2.mean(), l2.std()

(tensor(0.5661), tensor(0.8339))

好多了！现在我们的权重已经正确初始化了，我们可以定义我们的整个模型了：

In [ ]:
def model(x):
    l1 = lin(x, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

这是前向传播。现在唯一剩下的就是使用损失函数将我们的输出与我们拥有的标签（在这个例子中是随机数字）进行比较。在这种情况下，我们将使用均方误差。（这是一个玩具问题，这是接下来计算梯度时最简单的损失函数。）

唯一的微妙之处在于我们的输出和目标的形状并不完全相同——经过模型处理后，我们得到的输出是这样的：

In [ ]:
out = model(x)
out.shape

torch.Size([200, 1])

为了消除这个尾部的1维度，我们使用`squeeze`函数：

In [ ]:
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

现在我们准备好计算我们的损失了：

In [ ]:
loss = mse(out, y)

前向传播的内容就这些了——现在让我们看看梯度。

### 梯度和后向传播

我们已经看到PyTorch通过一个神奇的调用`loss.backward`为我们计算了所有需要的梯度，但让我们探索一下幕后发生了什么。

现在是我们计算损失相对于我们模型所有权重的梯度的部分，也就是`w1`、`b1`、`w2`和`b2`中的所有浮点数。为此，我们需要一点数学知识——特别是*链式法则*。这是微积分中的一个规则，指导我们如何计算复合函数的导数：

$$(g \circ f)'(x) = g'(f(x)) f'(x)$$

> j: 我发现这种表示法很难理解，所以我喜欢这样想：如果`y = g(u)`且`u=f(x)`；那么`dy/dx = dy/du * du/dx`。这两种表示法意思相同，所以用你觉得方便的那种。

我们的损失是不同函数的一个大组合：均方误差（这反过来又是均值和平方的组合），第二层线性层，ReLU和第一层线性层。例如，如果我们想要损失相对于`b2`的梯度，并且我们的损失由以下定义：

```
loss = mse(out,y) = mse(lin(l2, w2, b2), y)
```

链式法则告诉我们我们有：
$$\frac{\text{d} loss}{\text{d} b_{2}} = \frac{\text{d} loss}{\text{d} out} \times \frac{\text{d} out}{\text{d} b_{2}} = \frac{\text{d}}{\text{d} out} mse(out, y) \times \frac{\text{d}}{\text{d} b_{2}} lin(l_{2}, w_{2}, b_{2})$$

为了计算损失相对于`b2`的梯度，我们首先需要损失相对于我们的输出`out`的梯度。如果我们想要损失相对于`w2`的梯度，情况也是一样。然后，为了得到损失相对于`b1`或`w1`的梯度，我们需要损失相对于`l1`的梯度，这反过来又需要损失相对于`l2`的梯度，这将需要损失相对于`out`的梯度。

所以为了计算我们更新所需的所有梯度，我们需要从模型的输出开始，逐层向后工作，一层接一层——这就是为什么这一步被称为*反向传播*。我们可以通过让我们实现的每个函数（`relu`、`mse`、`lin`）提供其反向步骤来自动化它：即如何从损失相对于输出的梯度推导出损失相对于输入的梯度。

在这里，我们在每个张量的属性中填充这些梯度，有点像PyTorch对`.grad`的处理。

首先是损失相对于我们模型输出（即损失函数的输入）的梯度。我们撤销了在`mse`中所做的`squeeze`，然后我们使用给出`x^{2}`导数的公式：`2x`。均值的导数只是`1/n`，其中`n`是我们输入中元素的数量：

In [ ]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

对于ReLU和我们线性层的梯度，我们使用损失相对于输出的梯度（在`out.g`中），并应用链式法则来计算损失相对于输入的梯度（在`inp.g`中）。链式法则告诉我们`inp.g = relu'(inp) * out.g`。ReLU的导数要么是0（当输入为负），要么是1（当输入为正），所以这给我们带来了：

In [ ]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

计算线性层中损失相对于输入、权重和偏置的梯度的方案是相同的：

In [ ]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    w.g = inp.t() @ out.g
    b.g = out.g.sum(0)

我们不会过多停留在定义它们的数学公式上，因为它们对我们的目的来说并不重要，但如果你对这个话题感兴趣，一定要查看Khan Academy的优秀微积分课程。

### 侧边栏：SymPy

SymPy 是一个用于符号计算的库，在处理微积分问题时非常有用。根据[文档](https://docs.sympy.org/latest/tutorial/intro.html)：

> : 符号计算处理数学对象的符号化计算。这意味着数学对象被精确地表示，而不是近似地，并且包含未评估变量的数学表达式以符号形式保留。

要进行符号计算，我们首先定义一个*符号*，然后进行计算，如下所示：

In [ ]:
from sympy import symbols,diff
sx,sy = symbols('sx sy')
diff(sx**2, sx)

2*sx

在这里，SymPy已经为我们求出了`x**2`的导数！它可以求复杂复合表达式的导数，简化和分解方程，以及更多。如今，真的没有太多理由让人们手动进行微积分计算——对于计算梯度，PyTorch为我们做了这些，而对于展示方程，SymPy为我们做了这些！

### 结束侧边栏

一旦我们定义了这些函数，我们就可以使用它们来编写后向传播。由于每个梯度都会自动填充到正确的张量中，我们不需要在任何地方存储这些`_grad`函数的结果——我们只需要按照前向传播的逆序执行它们，以确保在每个函数中`out.g`都存在：

In [ ]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

现在我们可以在`w1.g`、`b1.g`、`w2.g`和`b2.g`中访问我们模型参数的梯度。

我们已经成功定义了我们的模型——现在让我们让它更像一个PyTorch模块。

### 重构模型

我们使用的三个函数有两个相关联的函数：前向传播和后向传播。我们不是分别编写它们，而是可以创建一个类将它们一起封装。这个类还可以为后向传播存储输入和输出。这样，我们只需要调用`backward`：

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

`__call__`是Python中的一个魔术名称，它将使我们的类变得可调用。当我们输入`y = Relu()(x)`时，将执行这个函数。我们可以对我们的线性层和均方误差损失做同样的事情：

In [ ]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        x = (self.inp.squeeze()-self.targ).unsqueeze(-1)
        self.inp.g = 2.*x/self.targ.shape[0]

然后我们可以把所有东西放在一个模型中，我们可以用我们的张量`w1`、`b1`、`w2`、`b2`来初始化它：

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

这种重构和将事物注册为我们模型的层的真正好处在于，前向和后向传播现在变得非常容易编写。如果我们想要实例化我们的模型，我们只需要写：

In [ ]:
model = Model(w1, b1, w2, b2)

然后前向传播可以通过以下方式执行：

In [ ]:
loss = model(x, y)

后向传播可以通过以下方式执行：

In [ ]:
model.backward()

### 转向PyTorch

我们编写的`Lin`、`Mse`和`Relu`类有很多共同点，所以我们可以让他们都继承自同一个基类：

In [ ]:
class LayerFunction():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self):  raise Exception('not implemented')
    def bwd(self):      raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

然后我们只需要在每个子类中实现`forward`和`bwd`：

In [ ]:
class Relu(LayerFunction):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(LayerFunction):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = out.g.sum(0)

In [ ]:
class Mse(LayerFunction):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): 
        inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

我们模型的其余部分可以和之前一样。这越来越接近PyTorch所做的。我们需要微分的每个基本函数都被写成一个具有`forward`和`backward`方法的`torch.autograd.Function`对象。然后PyTorch将跟踪我们所做的任何计算，以便能够正确运行后向传播，除非我们将张量的`requires_grad`属性设置为`False`。

编写这些函数（几乎）和编写我们最初的类一样容易。不同之处在于我们选择保存什么以及放入上下文变量中（这样我们可以确保不保存我们不需要的任何东西），并在后向传播中返回梯度。很少需要编写自己的`Function`，但如果你曾经需要一些特殊的功能或者想要调整常规函数的梯度，以下是如何编写一个：

In [ ]:
from torch.autograd import Function

class MyRelu(Function):
    @staticmethod
    def forward(ctx, i):
        result = i.clamp_min(0.)
        ctx.save_for_backward(i)
        return result
    
    @staticmethod
    def backward(ctx, grad_output):
        i, = ctx.saved_tensors
        return grad_output * (i>0).float()

构建一个利用这些`Function`s的更复杂模型的结构是`torch.nn.Module`。这是所有模型的基础结构，到目前为止你看到的所有神经网络都继承自这个类。它主要帮助注册所有可训练的参数，正如我们所看到的，这些参数可以在训练循环中使用。

要实现一个`nn.Module`，你只需要：

- 在初始化时首先调用超类`__init__`。
- 将模型的任何参数定义为具有`nn.Parameter`的属性。
- 定义一个返回模型输出的`forward`函数。

作为一个例子，这里是从头开始的线性层：

In [ ]:
import torch.nn as nn

class LinearLayer(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(n_out, n_in) * sqrt(2/n_in))
        self.bias = nn.Parameter(torch.zeros(n_out))
    
    def forward(self, x): return x @ self.weight.t() + self.bias

如你所见，这个类会自动跟踪已定义的参数：

In [ ]:
lin = LinearLayer(10,2)
p1,p2 = lin.parameters()
p1.shape,p2.shape

(torch.Size([2, 10]), torch.Size([2]))

正是由于`nn.Module`的这个特性，我们可以简单地说`opt.step()`，让优化器遍历参数并更新每一个。

请注意，在PyTorch中，权重被存储为一个`n_out x n_in`的矩阵，这就是为什么我们在前向传播中有转置。

通过使用PyTorch中的线性层（它也使用Kaiming初始化），我们在本章中构建的模型可以这样写：

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out))
        self.loss = mse
        
    def forward(self, x, targ): return self.loss(self.layers(x).squeeze(), targ)

fastai提供了自己的`Module`变体，它与`nn.Module`相同，但不要求你调用`super().__init__()`（它会自动为你做这个）：

In [ ]:
class Model(Module):
    def __init__(self, n_in, nh, n_out):
        self.layers = nn.Sequential(
            nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out))
        self.loss = mse
        
    def forward(self, x, targ): return self.loss(self.layers(x).squeeze(), targ)

在最后一章中，我们将从这样一个模型开始，看看如何从头构建一个训练循环，并重构它，使其成为我们在前几章中使用的形式。

## 总结

在本章中，我们探讨了深度学习的基础，从矩阵乘法开始，然后逐步实现神经网络的前向和后向传播。然后我们重构了我们的代码，展示了PyTorch在幕后的工作原理。

以下是一些需要记住的事情：

- 神经网络基本上是一堆矩阵乘法，中间夹杂着非线性。
- Python运行速度慢，所以为了编写快速的代码，我们必须向量化它，并利用逐元素算术和广播等技术。
- 如果从尾部开始向前的维度匹配（如果它们相同，或者其中一个是1），那么两个张量就是可广播的。为了使张量可广播，我们可能需要使用`unsqueeze`或`None`索引添加大小为1的维度。
- 正确初始化神经网络对于开始训练至关重要。当我们有ReLU非线性时，应该使用Kaiming初始化。
- 后向传播是多次应用链式法则，计算我们模型输出的梯度，并逐层向后传递。
- 当继承`nn.Module`（如果不使用fastai的`Module`）时，我们必须在我们的`__init__`方法中调用超类`__init__`方法，并且我们必须定义一个`forward`函数，它接受一个输入并返回期望的结果。

## 问卷调查

1. 编写Python代码实现单个神经元。
2. 编写Python代码实现ReLU。
3. 用矩阵乘法的方式编写密集层的Python代码。
4. 使用列表推导式和Python内置功能，用纯Python编写密集层的代码。
5. 什么是层的“隐藏大小”？
6. PyTorch中的`t`方法有什么作用？
7. 为什么用纯Python编写的矩阵乘法非常慢？
8. 在`matmul`中，为什么`ac==br`？
9. 在Jupyter Notebook中，如何测量单个单元格执行所需的时间？
10. “逐元素算术”是什么？
11. 编写PyTorch代码，测试`a`的每个元素是否大于`b`的对应元素。
12. 什么是秩为0的张量？如何将其转换为普通的Python数据类型？
13. 这将返回什么，为什么？`tensor([1,2]) + tensor([1])`
14. 这将返回什么，为什么？`tensor([1,2]) + tensor([1,2,3])`
15. 逐元素算术如何帮助我们加速`matmul`？
16. 广播规则是什么？
17. `expand_as`是什么？展示一个如何使用它来匹配广播结果的例子。
18. `unsqueeze`如何帮助我们解决某些广播问题？
19. 我们如何使用索引来执行与`unsqueeze`相同的操作？
20. 我们如何展示张量使用的内存的实际内容？
21. 当向大小为3的向量添加到大小为3×3的矩阵时，向量的元素是添加到矩阵的每一行还是每一列？（确保通过在笔记本中运行此代码来检查你的答案。）
22. 广播和`expand_as`是否会导致内存使用增加？为什么或为什么不？
23. 使用爱因斯坦求和实现`matmul`。
24. 在einsum的左侧，重复的索引字母代表什么？
25. 爱因斯坦求和记号的三条规则是什么？为什么？
26. 神经网络的前向传播和后向传播是什么？
27. 为什么我们需要在前向传播中存储中间层计算的一些激活值？
28. 激活值的标准差远离1有什么缺点？
29. 权重初始化如何帮助避免这个问题？
30. 初始化权重的公式是什么，以便我们得到普通线性层的标准差为1，以及ReLU之后的线性层？
31. 为什么有时我们需要在损失函数中使用`squeeze`方法？
32. `squeeze`方法的参数做什么？为什么即使PyTorch不要求，包含这个参数可能也很重要？
33. “链式法则”是什么？展示本章中呈现的两种形式之一的方程。
34. 使用链式法则计算`mse(lin(l2, w2, b2), y)`的梯度。
35. ReLU的梯度是什么？用数学或代码展示。（你不需要记住这个——尝试根据你对函数形状的了解来弄清楚。）
36. 在后向传播中，我们需要按什么顺序调用`*_grad`函数？为什么？
37. `__call__`是什么？
38. 编写`torch.autograd.Function`时，我们必须实现哪些方法？
39. 从头开始编写`nn.Linear`，并测试它是否工作。
40. `nn.Module`和fastai的`Module`之间有什么区别？

### 进一步研究

1. 将ReLU实现为`torch.autograd.Function`，并用它训练一个模型。
2. 如果你对数学感兴趣，找出线性层的梯度在数学符号中是什么。将其映射到我们在本章中看到的实现。
3. 了解PyTorch中的`unfold`方法，并结合矩阵乘法实现你自己的2D卷积函数。然后训练一个使用它的CNN。
4. 使用NumPy而不是PyTorch实现本章中的所有内容。